In [1]:
import pyspark
import os
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StructType, StructField, StringType, DateType

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'
ss = SparkSession.builder.appName("l2").master("local[*]").getOrCreate()
ss



24/03/28 14:18:53 WARN Utils: Your hostname, pavel-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.70.130 instead (on interface ens33)
24/03/28 14:18:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/pavel/.ivy2/cache
The jars for the packages stored in: /home/pavel/.ivy2/jars
:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-39c6ecf0-59bb-40a5-a4cf-315346c6e001;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.13.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 637ms :: artifacts dl 26ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.13.0 from central in [defau

In [3]:
read_csv = ss.read.csv("programming-languages.csv")
csv_list = [str(x[0]) for x in read_csv.collect()]
csv_list[:5]

['name', 'A# .NET', 'A# (Axiom)', 'A-0 System', 'A+']

In [4]:
posts = ss.read.format("xml").options(rowTag="row").load('posts_sample.xml')
posts.take(5)

[Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 20, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 8, 1, 2, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 5, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 5, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUser

In [5]:
def check_lang(x):
    tag = next((language for language in csv_list if f"<{language.lower()}>" in x._Tags.lower()), None)
    return (x._Id, tag) if tag else None

def check_date(x, year):
    creation_date = x._CreationDate
    return creation_date.year == year and creation_date.month >= 1 and creation_date.month <= 12

def aggregate_function(x, y):
    return x + 1

def merge_function(x1, x2):
    return x1 + x2

result = {}
for y in range(2010, 2020):
  result[y] = posts.rdd.filter(lambda x: x._Tags is not None and check_date(x, y)).map(check_lang).filter(lambda x: x is not None)\
      .keyBy(lambda x: x[1]).aggregateByKey(0, aggregate_function, merge_function,).sortBy(lambda x: x[1], ascending=False).toDF()
  result[y] = result[y].select(col("_1").alias("Programming_language"), col("_2").alias(f"Number_of_mentions_in_{y}")).limit(10)
  result[y].show()

+--------------------+--------------------------+
|Programming_language|Number_of_mentions_in_2010|
+--------------------+--------------------------+
|                Java|                        52|
|          JavaScript|                        44|
|                 PHP|                        42|
|              Python|                        25|
|         Objective-C|                        23|
|                   C|                        20|
|                Ruby|                        11|
|              Delphi|                         7|
|                   R|                         3|
|                Bash|                         3|
+--------------------+--------------------------+



+--------------------+--------------------------+
|Programming_language|Number_of_mentions_in_2011|
+--------------------+--------------------------+
|                 PHP|                        97|
|                Java|                        92|
|          JavaScript|                        82|
|              Python|                        35|
|         Objective-C|                        33|
|                   C|                        24|
|                Ruby|                        17|
|              Delphi|                         8|
|                Perl|                         8|
|                Bash|                         7|
+--------------------+--------------------------+



+--------------------+--------------------------+
|Programming_language|Number_of_mentions_in_2012|
+--------------------+--------------------------+
|                 PHP|                       136|
|          JavaScript|                       129|
|                Java|                       124|
|              Python|                        65|
|         Objective-C|                        45|
|                   C|                        27|
|                Ruby|                        25|
|                Bash|                         9|
|                   R|                         9|
|              MATLAB|                         6|
+--------------------+--------------------------+



+--------------------+--------------------------+
|Programming_language|Number_of_mentions_in_2013|
+--------------------+--------------------------+
|          JavaScript|                       196|
|                Java|                       191|
|                 PHP|                       173|
|              Python|                        87|
|         Objective-C|                        40|
|                   C|                        36|
|                Ruby|                        30|
|                   R|                        25|
|                Bash|                        11|
|               Scala|                        10|
+--------------------+--------------------------+



+--------------------+--------------------------+
|Programming_language|Number_of_mentions_in_2014|
+--------------------+--------------------------+
|          JavaScript|                       235|
|                Java|                       228|
|                 PHP|                       154|
|              Python|                       103|
|                   C|                        52|
|         Objective-C|                        49|
|                   R|                        28|
|                Ruby|                        20|
|              MATLAB|                        16|
|                Bash|                        13|
+--------------------+--------------------------+



+--------------------+--------------------------+
|Programming_language|Number_of_mentions_in_2015|
+--------------------+--------------------------+
|          JavaScript|                       270|
|                Java|                       208|
|                 PHP|                       147|
|              Python|                       119|
|                   R|                        43|
|                   C|                        38|
|         Objective-C|                        30|
|                Ruby|                        21|
|              MATLAB|                        16|
|               Scala|                        13|
+--------------------+--------------------------+



+--------------------+--------------------------+
|Programming_language|Number_of_mentions_in_2016|
+--------------------+--------------------------+
|          JavaScript|                       272|
|                Java|                       179|
|              Python|                       141|
|                 PHP|                       126|
|                   R|                        50|
|                   C|                        32|
|                Ruby|                        21|
|                Bash|                        16|
|               Scala|                        16|
|              MATLAB|                        15|
+--------------------+--------------------------+



+--------------------+--------------------------+
|Programming_language|Number_of_mentions_in_2017|
+--------------------+--------------------------+
|          JavaScript|                       244|
|                Java|                       204|
|              Python|                       185|
|                 PHP|                       122|
|                   R|                        53|
|                   C|                        24|
|         Objective-C|                        19|
|                Ruby|                        16|
|          TypeScript|                        14|
|          PowerShell|                        14|
+--------------------+--------------------------+



+--------------------+--------------------------+
|Programming_language|Number_of_mentions_in_2018|
+--------------------+--------------------------+
|              Python|                       214|
|          JavaScript|                       196|
|                Java|                       145|
|                 PHP|                        99|
|                   R|                        63|
|                   C|                        24|
|               Scala|                        22|
|          TypeScript|                        21|
|          PowerShell|                        13|
|                Bash|                        12|
+--------------------+--------------------------+



+--------------------+--------------------------+
|Programming_language|Number_of_mentions_in_2019|
+--------------------+--------------------------+
|              Python|                       162|
|          JavaScript|                       131|
|                Java|                        95|
|                 PHP|                        59|
|                   R|                        36|
|                   C|                        14|
|                  Go|                         9|
|              MATLAB|                         9|
|              Kotlin|                         9|
|                Dart|                         9|
+--------------------+--------------------------+



In [7]:
for k, v in result.items():
    v.write.format("parquet").save(f"top_{k}")